In [ ]:
%load_ext autoreload
%autoreload 2
RootDIR = "/home/jw3514/Work/ASD_Circuits_CellType/"
import sys
sys.path.insert(1, RootDIR + 'src')
from ASD_Circuits import *

In [ ]:
W_ipsi = pd.read_excel("../dat/allen-mouse-conn/nature13186-s4.xlsx", sheet_name="W_ipsi")
PValue_ipsi = pd.read_excel("../dat/allen-mouse-conn/nature13186-s4.xlsx", sheet_name="PValue_ipsi")
W_contra = pd.read_excel("../dat/allen-mouse-conn/nature13186-s4.xlsx", sheet_name="W_contra")
PValue_contra = pd.read_excel("../dat/allen-mouse-conn/nature13186-s4.xlsx", sheet_name="PValue_contra")
W_ipsi = W_ipsi.set_index("Unnamed: 0")
PValue_ipsi = PValue_ipsi.set_index("Unnamed: 0")
W_contra = W_contra.set_index("Unnamed: 0")
PValue_contra = PValue_contra.set_index("Unnamed: 0")

In [ ]:
ConnMat_v2_acronym = W_ipsi.join(W_contra, how='left', lsuffix='_ipsi', rsuffix='_contra')

In [ ]:
W_ipsi[PValue_ipsi > 0.01] = 0
W_contra[PValue_contra > 0.01] = 0

In [ ]:
inter, union, ipsi_only, cont_only, NoConn = 0,0,0,0,0
Weights_ipsi, Weights_contra = [], []
for str_i in W_contra.index.values:
    for str_j in W_contra.columns.values:
        ipsi = W_ipsi.loc[str_i, str_j]
        cont = W_contra.loc[str_i, str_j]
        if ipsi > 0 or cont > 0:
            union += 1
        if ipsi > 0 and cont > 0:
            inter += 1
        if ipsi >0 and cont == 0:
            ipsi_only += 1
            Weights_ipsi.append(ipsi)
        if cont > 0 and ipsi == 0:
            cont_only += 1
            #print(acronym2name[str_i], acronym2name[str_j], cont)
            Weights_contra.append(cont)
        if ipsi ==0 and cont ==0:
            NoConn += 1
print(inter, union, ipsi_only, cont_only, NoConn)

In [ ]:
1395 * 2 + 1728 + 1056

In [ ]:
plt.hist(np.log10(Weights_ipsi), color='red', bins=10, alpha=0.5, density=1)
plt.hist(np.log10(Weights_contra), color='blue', bins=10, alpha=0.5, density=1)

In [ ]:
ontology = pd.read_csv("../dat/Other/ontology.csv")
#onotology = pd.read_csv("dat/voxel.csv")
#ontology["acronym"] = 
acronym2name = {}
for i, row in ontology.iterrows():
    acronym2name[row["acronym"]] = clean_name(row["safe_name"])

W_ipsi.columns = [acronym2name[x] for x in W_ipsi.columns.values]
W_ipsi.index = [acronym2name[x] for x in W_ipsi.index.values]

W_contra.columns = [acronym2name[x] for x in W_contra.columns.values]
W_contra.index = [acronym2name[x] for x in W_contra.index.values]

In [ ]:
ConnMat_v2 = W_ipsi.join(W_contra, how='left', lsuffix='_ipsi', rsuffix='_contra')
W_ipsi.columns = ["{}_ipsi".format(x) for x in W_ipsi.columns.values]
W_contra.columns = ["{}_contra".format(x) for x in W_contra.columns.values]
W_ipsi.to_csv(outD)

##### Get Distance

In [ ]:
RawDistance = pd.read_excel("../dat/allen-mouse-conn/41586_2014_BFnature13186_MOESM72_ESM.xlsx", 
                            index_col=0)
# Combined
DistanceMat_v2 = pd.DataFrame(data=np.zeros((213,426)), index=ConnMat_v2_acronym.index.values,
                             columns=ConnMat_v2_acronym.columns.values)
for str_i in ConnMat_v2_acronym.index.values:
    for str_j in ConnMat_v2_acronym.columns.values:
        dist = RawDistance.loc["{}_ipsi".format(str_i), str_j]
        DistanceMat_v2.loc[str_i, str_j] = dist
DistanceMat_v2.index = ConnMat_v2.index.values
DistanceMat_v2.columns = ConnMat_v2.columns.values
DistanceMat_v2.to_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.ipsi.contra.csv")

In [ ]:
# Seperate ipsi/contra
DistanceMat_ipsi = DistanceMat_v2.loc[:, W_ipsi.columns.values]
DistanceMat_contra = DistanceMat_v2.loc[:, W_contra.columns.values]

In [ ]:
DistanceMat_ipsi.to_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.ipsi.csv")
DistanceMat_contra.to_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.contra.csv")

In [ ]:
ipsi_dists = DistanceMat_ipsi.values.flatten()
contra_dists = DistanceMat_contra.values.flatten()
plt.scatter(ipsi_dists, contra_dists, s=1)
pearsonr(ipsi_dists, contra_dists)

In [ ]:
def MakeInfoMat_Distance(bins, Info, adjMat, distMat):
    OutMat = pd.DataFrame(data=np.zeros((adjMat.shape[0], adjMat.shape[1])), 
                          index=adjMat.index.values, columns=adjMat.columns.values)
    OutMat2 = pd.DataFrame(data=np.zeros((adjMat.shape[0], adjMat.shape[1])), 
                           index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        for node_j in adjMat.columns.values:
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                #print(node_i, node_j)
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        OutMat2.loc[node_i, node_j] = Info[i] #np.exp(-Info[i])
                        if conn > 0:
                            OutMat.loc[node_i, node_j] = Info[i]
    return OutMat, OutMat2

###### ipsi

In [ ]:
All_distances = DistanceMat_ipsi.values.flatten()
DistanceDeciles = np.percentile(All_distances, np.arange(0, 100, 10))
DistanceDeciles = np.append(DistanceDeciles, max(All_distances))
distances = DistanceMat_ipsi[W_ipsi>0]
distances = distances[~np.isnan(distances)]

In [ ]:
Counts_all, bins = np.histogram(All_distances, bins=DistanceDeciles)
Counts_conn, bins = np.histogram(distances, bins=DistanceDeciles)
r = range(len(bins)-1)
names = ["%.0f-%.0f"%(bins[i], bins[i+1]) for i in range(len(bins)-1)]
plt.figure(dpi=120)
barWidth = 0.85
plt.bar(r, Counts_all, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
plt.bar(r, Counts_conn, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
plt.xticks(r, names)
plt.xlabel("group")
plt.xticks(rotation = 45) 
plt.title("ipsi")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(12,6))

Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

Probs_ = 1 - np.array(Counts_conn/Counts_all)
Infos_ = -np.log2(Probs_)

l1 = ax1.plot(bins[:-1], Probs, c="black", marker='o', label="Connection Probability")
ax1.set_xlabel("Distance (mm)")
ax1.set_ylabel("Connection Probability")
ax1_ = ax1.twinx()
l2 = ax1_.plot(bins[:-1], Infos, marker='o', label="Shannon Information")
ax1_.set_ylabel("Infomation")
ax1.legend(lns, labs, loc=0)

l1 = ax2.plot(bins[:-1], Probs_, c="black", marker='o', label="Connection Probability")
ax2.set_xlabel("Distance (mm)")
ax2.set_ylabel("Connection Probability")
ax2_ = ax2.twinx()
l2 = ax2_.plot(bins[:-1], Infos_, marker='o', label="Shannon Information")
ax2_.set_ylabel("Infomation")

lns = l1 + l2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=0)
plt.tight_layout()

In [ ]:
OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v2/"

Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

InfoMat_Distance_V2, XX = MakeInfoMat_Distance(bins, Infos, 
                                ConnMat_v2, DistanceMat_v2)
InfoMat_Distance_V2.to_csv(OutDIR + "ScoreMat_Distance.ipsi.csv")
XX.to_csv(OutDIR + "ScoreMat_Distance.Entropy.ipsi.csv")

###### contra

In [ ]:
All_distances = DistanceMat_contra.values.flatten()
DistanceDeciles = np.percentile(All_distances, np.arange(0, 100, 10))
DistanceDeciles = np.append(DistanceDeciles, max(All_distances))
distances = DistanceMat_contra[W_contra>0]
distances = distances[~np.isnan(distances)]

In [ ]:
Counts_all, bins = np.histogram(All_distances, bins=DistanceDeciles)
Counts_conn, bins = np.histogram(distances, bins=DistanceDeciles)
r = range(len(bins)-1)
names = ["%.0f-%.0f"%(bins[i], bins[i+1]) for i in range(len(bins)-1)]
plt.figure(dpi=120)
barWidth = 0.85
plt.bar(r, Counts_all, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
plt.bar(r, Counts_conn, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
plt.xticks(r, names)
plt.xlabel("group")
plt.xticks(rotation = 45) 
plt.title("contra")
plt.show()

In [ ]:
OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v2/"

Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

InfoMat_Distance_V2, XX = MakeInfoMat_Distance(bins, Infos, 
                                ConnMat_v2, DistanceMat_v2)
InfoMat_Distance_V2.to_csv(OutDIR + "ScoreMat_Distance.contra.csv")
XX.to_csv(OutDIR + "ScoreMat_Distance.Entropy.contra.csv")

##### Joint Distribution

In [ ]:
"""
for str_i in W_ipsi.index.values:
    for str_j in W_ipsi.index.values:
        target_ipsi = "{}_ipsi".format(str_j)
        target_contra = "{}_contra".format(str_j)
        
        Dist_ipsi = DistanceMat_ipsi.loc[str_i, target_ipsi]
"""

In [ ]:
DistanceMat_ipsi

In [ ]:
#1/0
All_distances = DistanceMat_ipsi.values.flatten()
DistanceDeciles = np.percentile(All_distances, np.arange(0, 100, 10))
DistanceDeciles = np.append(DistanceDeciles, max(All_distances))
distances = DistanceMat_ipsi[W_ipsi>0 and W_contra]
distances = distances[~np.isnan(distances)]

##### padding

In [ ]:
NoSource, NoTarget = [], []
for STR in ConnMat_v2.index:
    Source = conn_p005.loc[STR, :].values
    #Target = conn_p005.loc[:, STR].values
    #print(Source)
    _source = np.where(Source!=0)[0]
    _target = np.where(Target!=0)[0]
    if len(_target) == 0:
        NoTarget.append(STR)
    if len(_source)==0:
        NoSource.append(STR)
print(NoSource, NoTarget)

In [ ]:
NoTargets = []
for STR in ConnMat_v2.index:
    Targets = ConnMat_v2.loc[STR, :]
    _target = np.where(Targets!=0)[0]
    if len(_target) == 0:
        NoTargets.append(STR)
print(NoTargets)

##### Save Matrix

In [ ]:

SaveDir = "../dat/allen-mouse-conn/ScoreingMat_jw_v2/"
ConnMat_v2.to_csv("{}/ScoreMat.Weight.csv".format(SaveDir))
W_ipsi.to_csv("{}/ScoreMat.Weight.ipsi.csv".format(SaveDir))
W_contra.to_csv("{}/ScoreMat.Weight.contra.csv".format(SaveDir))

### Connection distances data

In [ ]:
RawDistance.head(2)

In [ ]:
ConnMat_v2_acronym.head(2)

In [ ]:
RawDistance.loc["AAA_ipsi", "AAA_ipsi"]

In [ ]:
DistanceMat_v2 = pd.DataFrame(data=np.zeros((213,426)), index=ConnMat_v2_acronym.index.values,
                             columns=ConnMat_v2_acronym.columns.values)

In [ ]:
# Rename columns and index
for str_i in ConnMat_v2_acronym.index.values:
    for str_j in ConnMat_v2_acronym.columns.values:
        dist = RawDistance.loc["{}_ipsi".format(str_i), str_j]
        DistanceMat_v2.loc[str_i, str_j] = dist

In [ ]:
DistanceMat_v2.index = ConnMat_v2.index.values
DistanceMat_v2.columns = ConnMat_v2.columns.values

In [ ]:
DistanceMat_v2

In [ ]:
DistanceMat_v2.to_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.ipsi.contra.csv")

In [ ]:
All_distances = DistanceMat_v2.values.flatten()
DistanceDeciles = np.percentile(All_distances, np.arange(0, 100, 10))
DistanceDeciles = np.append(DistanceDeciles, max(All_distances))

In [ ]:
distances = DistanceMat_v2[ConnMat_v2>0]
distances = distances[~np.isnan(distances)]

In [ ]:
Counts_all, bins = np.histogram(All_distances, bins=DistanceDeciles)
Counts_conn, bins = np.histogram(distances, bins=DistanceDeciles)
r = range(len(bins)-1)
names = ["%.0f-%.0f"%(bins[i], bins[i+1]) for i in range(len(bins)-1)]
barWidth = 0.85
plt.bar(r, Counts_all, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
plt.bar(r, Counts_conn, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
plt.xticks(r, names)
plt.xlabel("group")
plt.xticks(rotation = 45) 
plt.show()

In [ ]:
Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

fig,ax = plt.subplots(dpi=120)
ax.plot(bins[:-1], Probs, c="black", marker='o', label="Connection Probability")
ax.set_xlabel("Distance (mm)")
ax.set_ylabel("Connection Probability")
ax2 = ax.twinx()
ax2.plot(bins[:-1], Infos, marker='o', label="Shannon Information")
ax2.set_ylabel("Infomation")

In [ ]:
def MakeInfoMat_Distance(bins, Info, adjMat, distMat):
    OutMat = pd.DataFrame(data=np.zeros((adjMat.shape[0], adjMat.shape[1])), 
                          index=adjMat.index.values, columns=adjMat.columns.values)
    OutMat2 = pd.DataFrame(data=np.zeros((adjMat.shape[0], adjMat.shape[1])), 
                           index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        for node_j in adjMat.columns.values:
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                #print(node_i, node_j)
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        OutMat2.loc[node_i, node_j] = Info[i] #np.exp(-Info[i])
                        if conn > 0:
                            OutMat.loc[node_i, node_j] = Info[i]
    return OutMat, OutMat2

OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v2/"
InfoMat_Distance_V2, XX = MakeInfoMat_Distance(bins, Infos, 
                                ConnMat_v2, DistanceMat_v2)
InfoMat_Distance_V2.to_csv(OutDIR + "ScoreMat_Distance.csv")
XX.to_csv(OutDIR + "ScoreMat_Distance.Entropy.csv")

In [ ]:
adj_short, adj_long = ConnectomeSeperation_Distance(InfoMat_Distance_V2, DistanceMat_v2)
adj_short.to_csv(OutDIR + "ScoreMat_Distance.short.csv")
adj_long.to_csv(OutDIR + "ScoreMat_Distance.long.csv")

In [ ]:
# 
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")
ASD_BiasDF.head(2)

In [ ]:
adj_mat = ConnMat_v2
InfoMat = InfoMat_Distance_V2
ProbMat1 = np.exp2(-InfoMat)
ProbMat1[ProbMat1==1] = 0
ProbMat2 = 1-ProbMat1

InfoMat_long = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat_Distance.long.csv", index_col=0)
ProbMat1_long = np.exp2(-InfoMat_long)
ProbMat1_long[ProbMat1_long==1] = 0
ProbMat2_long = 1-ProbMat1_long

InfoMat_short = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat_Distance.short.csv", index_col=0)
ProbMat1_short = np.exp2(-InfoMat_short)
ProbMat1_short[ProbMat1_short==1] = 0
ProbMat2_short = 1-ProbMat1_short


In [ ]:
Ncont = 100

contDir = "../dat/Unionize_bias/SubSampleSib/"
topNs = list(range(200, 5, -5))
ASD_Cont_SI_SCores_All = []
ASD_Cont_SI_SCores_Short = []
ASD_Cont_SI_SCores_Long = []
for i, file in enumerate(os.listdir(contDir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(contDir + file, index_col="STR")
    cont_scores = []
    cont_scores_short = []
    cont_scores_long = []
    for topN in topNs:
        top_strs = df.index.values[:topN]
        score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
        score_short = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
        score_long = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
        cont_scores.append(score)
        cont_scores_short.append(score_short)
        cont_scores_long.append(score_long)
    cont_scores = np.array(cont_scores)
    cont_scores_short = np.array(cont_scores_short)
    cont_scores_long = np.array(cont_scores_long)
    ASD_Cont_SI_SCores_All.append(cont_scores)
    ASD_Cont_SI_SCores_Short.append(cont_scores_short)
    ASD_Cont_SI_SCores_Long.append(cont_scores_long)
    if i >= Ncont -1:
        break
ASD_Cont_SI_SCores = np.array(ASD_Cont_SI_SCores)
ASD_Cont_SI_SCores_Short = np.array(ASD_Cont_SI_SCores_Short)
ASD_Cont_SI_SCores_Long = np.array(ASD_Cont_SI_SCores_Long)

In [ ]:
asd_scores = []
asd_scores_short = []
asd_scores_long = []
for topN in topNs:
    top_strs = ASD_BiasDF.index.values[:topN]
    score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
    score_short = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
    score_long = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
    asd_scores.append(score)
    asd_scores_short.append(score_short)
    asd_scores_long.append(score_long)
asd_scores = np.array(asd_scores)
asd_scores_short = np.array(asd_scores_short)
asd_scores_long = np.array(asd_scores_long)

In [ ]:
plt.plot(topNs, ASD_SI)
plt.plot(topNs, ASD_Cont_SI_SCores.mean(axis=0))

In [ ]:
plt.plot(topNs, asd_scores_short)
plt.plot(topNs, ASD_Cont_SI_SCores_Short.mean(axis=0))

In [ ]:
plt.plot(topNs, asd_scores_long)
plt.plot(topNs, ASD_Cont_SI_SCores_Long.mean(axis=0))

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(8,3))
cont_xx = ASD_SI - ASD_Cont_SI_SCores
#cont = Cont_DistanceLong.nanmean(axis=0)
cont = np.nanmean(ASD_Cont_SI_SCores, axis=0)
#ax.plot(topNs, ASD_DistanceLong, color="blue", marker="o", markersize=2, lw=0.1,
#                     ls="dashed", label="ASD")
#ax.plot(topNs, Sib_DistanceLong, color="orange", marker="o", markersize=2, lw=0.1,
#                     ls="dashed", label="Sibling")


#lower = np.percentile(Cont_Distance, 2.5, axis=0)
#upper = np.percentile(Cont_Distance, 97.5, axis=0)
mean = cont_xx.mean(axis=0)
lower = np.percentile(cont_xx, 50-34.1, axis=0)
upper = np.percentile(cont_xx, 50+34.1, axis=0)
ax.errorbar(topNs, mean, color="grey", marker="o", markersize=1.5, lw=0.5,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="Info-Diff")

#ax.set_title("Connection Info")
ax.grid(True)
ax.set_xlabel("Structure Rank")
ax.set_ylabel("Circuit Score")
#ax.set_ylim((0.2, 0.9))
plt.legend()

In [ ]:
plt.figure(dpi=120)
plt.plot(topNs, ASD_SI-ASD_Cont_SI_SCores.mean(axis=0))
plt.grid(True)

In [ ]:
topN = 46

Ncont = 10000
contDir = "../dat/Unionize_bias/SubSampleSib/"

topN_bias = []
cont_topN_Scores = []
cont_topN_Scores_short = []
cont_topN_Scores_long = []

for i, file in enumerate(os.listdir(contDir)):
    if not file.startswith("cont.bias."):
        continue
    df = pd.read_csv(contDir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    
    cont_score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
    cont_score_short = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
    cont_score_long = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
    cont_topN_Scores.append(cont_score)
    cont_topN_Scores_short.append(cont_score_short)
    cont_topN_Scores_long.append(cont_score_long)
    #if cont_score_long > 0.65:
    #    print(RegionDistributionsList(top_strs))
    if i > Ncont:
        break
print(len(cont_topN_Scores))

asd_score = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, ProbMat1, ProbMat2)
asd_score_short = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, 
                                              ProbMat1_short, ProbMat2_short)
asd_score_long = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, 
                                             ProbMat1_long, ProbMat2_long)


In [ ]:
fig, axs = plt.subplots(1,3, dpi=480, figsize=(12,3))
PlotPermutationP(cont_topN_Scores, asd_score, axs[0],
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")
PlotPermutationP(cont_topN_Scores_short, asd_score_short, axs[1],
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score Short", 
                     dist_label="Sibling", bar_label="ASD")
PlotPermutationP(cont_topN_Scores_long, asd_score_long, axs[2],
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score Long", 
                     dist_label="Sibling", bar_label="ASD")

In [ ]:
topN = 90

Ncont = 10000
contDir = "../dat/Unionize_bias/SubSampleSib/"

topN_bias = []
cont_topN_Scores = []
cont_topN_Scores_short = []
cont_topN_Scores_long = []



for i, file in enumerate(os.listdir(contDir)):
    if not file.startswith("cont.bias."):
        continue
    df = pd.read_csv(contDir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    
    cont_score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
    #cont_score_short = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
    #cont_score_long = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
    cont_topN_Scores.append(cont_score)
    #cont_topN_Scores_short.append(cont_score_short)
    #cont_topN_Scores_long.append(cont_score_long)
    #if cont_score_long > 0.65:
    #    print(RegionDistributionsList(top_strs))
    if i > Ncont:
        break
print(len(cont_topN_Scores))

asd_score = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, ProbMat1, ProbMat2)
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
PlotPermutationP(cont_topN_Scores, asd_score, ax,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

In [ ]:
topN = 40

Ncont = 10000
contDir = "../dat/Unionize_bias/SubSampleSib/"

topN_bias = []
cont_topN_Scores = []
cont_topN_Scores_short = []
cont_topN_Scores_long = []



for i, file in enumerate(os.listdir(contDir)):
    if not file.startswith("cont.bias."):
        continue
    df = pd.read_csv(contDir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    
    cont_score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
    #cont_score_short = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
    #cont_score_long = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
    cont_topN_Scores.append(cont_score)
    #cont_topN_Scores_short.append(cont_score_short)
    #cont_topN_Scores_long.append(cont_score_long)
    #if cont_score_long > 0.65:
    #    print(RegionDistributionsList(top_strs))
    if i > Ncont:
        break
print(len(cont_topN_Scores))

asd_score = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, ProbMat1, ProbMat2)
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
PlotPermutationP(cont_topN_Scores, asd_score, ax,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

In [ ]:
topN = 20

Ncont = 10000
contDir = "../dat/Unionize_bias/SubSampleSib/"

topN_bias = []
cont_topN_Scores = []
cont_topN_Scores_short = []
cont_topN_Scores_long = []

asd_score = ScoreCircuit_SI_ipsi_contra(ASD_BiasDF.head(topN).index.values, adj_mat, ProbMat1, ProbMat2)

for i, file in enumerate(os.listdir(contDir)):
    if not file.startswith("cont.bias."):
        continue
    df = pd.read_csv(contDir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    
    cont_score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
    #cont_score_short = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
    #cont_score_long = ScoreCircuit_v7(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
    cont_topN_Scores.append(cont_score)
    #cont_topN_Scores_short.append(cont_score_short)
    #cont_topN_Scores_long.append(cont_score_long)
    #if cont_score_long > 0.65:
    #    print(RegionDistributionsList(top_strs))
    if i > Ncont:
        break
print(len(cont_topN_Scores))

fig, ax = plt.subplots(dpi=480, figsize=(6,6))
PlotPermutationP(cont_topN_Scores, asd_score, ax,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

In [ ]:
topNs = np.arange(200, 5, -1)
ASD_DistanceLong = np.load("RankScores/RankScore.DistanceLong.mutsim.ASD.S1.npy")
Cont_DistanceLong = np.load("RankScore2/RankScore.DistanceLong.mutsim.Cont.S1.npy")
Sib_DistanceLong = PlotXXX(SibDF, topNs, adj_mat=adj_mat, ProbMat1=ProbMat1_long, ProbMat2=ProbMat2_long)

# SI scores 

### Ipsi Contra Seperated

In [ ]:
adj_mat_ipsi = pd.read_csv(
    "../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat.Weight.ipsi.csv",
                          index_col=0)
InfoMat_ipsi = pd.read_csv(
    "../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat_Distance.Entropy.ipsi.csv",
                     index_col=0)
ProbMat1_ipsi = np.exp2(-InfoMat_ipsi)
ProbMat1_ipsi[ProbMat1_ipsi==1] = 0
ProbMat2_ipsi = 1-ProbMat1_ipsi

In [ ]:
Ncont = 100
contDir = "../dat/Unionize_bias/SubSampleSib/"
topNs = list(range(200, 5, -5))

def SI_Rank_profile_Case(ASD_BiasDF, adj_mat, ProbMat1, ProbMat2, topNs):
    asd_scores = []
    #asd_scores_short = []
    #asd_scores_long = []
    for topN in topNs:
       # print(topN, ASD_BiasDF.index.values[:topN])
        top_strs = ASD_BiasDF.index.values[:topN]
        score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1, ProbMat2)
        #score_short = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
        #score_long = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
        asd_scores.append(score)
        #asd_scores_short.append(score_short)
        #asd_scores_long.append(score_long)
    asd_scores = np.array(asd_scores)
    #asd_scores_short = np.array(asd_scores_short)
    #asd_scores_long = np.array(asd_scores_long)
    return asd_scores

def SI_Rank_Profile_Ctrl(contDir, adj_mat, ProbMat1, ProbMat2, topNs, Ncont=100):
    ASD_Cont_SI_SCores_All = []
    #ASD_Cont_SI_SCores_Short = []
    #ASD_Cont_SI_SCores_Long = []
    for i, file in enumerate(os.listdir(contDir)):
        if file.startswith("cont.genes"):
            continue
        df = pd.read_csv(contDir + file, index_col="STR")
        cont_scores = []
        #cont_scores_short = []
        #cont_scores_long = []
        for topN in topNs:
            top_strs = df.index.values[:topN]
            score = ScoreCircuit_SI_ipsi_contra(
                top_strs, adj_mat, ProbMat1, ProbMat2)
            #score_short = ScoreCircuit_SI_ipsi_contra(
            #   top_strs, adj_mat, ProbMat1_short, ProbMat2_short)
            #score_long = ScoreCircuit_SI_ipsi_contra(
            #    top_strs, adj_mat, ProbMat1_long, ProbMat2_long)
            cont_scores.append(score)
            #cont_scores_short.append(score_short)
            #cont_scores_long.append(score_long)
        cont_scores = np.array(cont_scores)
        #cont_scores_short = np.array(cont_scores_short)
        #cont_scores_long = np.array(cont_scores_long)
        ASD_Cont_SI_SCores_All.append(cont_scores)
        #ASD_Cont_SI_SCores_Short.append(cont_scores_short)
        #ASD_Cont_SI_SCores_Long.append(cont_scores_long)
        if i >= Ncont -1:
            break
    ASD_Cont_SI_SCores_All = np.array(ASD_Cont_SI_SCores_All)
    #ASD_Cont_SI_SCores_Short = np.array(ASD_Cont_SI_SCores_Short)
    #ASD_Cont_SI_SCores_Long = np.array(ASD_Cont_SI_SCores_Long)
    return ASD_Cont_SI_SCores_All

In [ ]:
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")
Ncont = 100
contDir = "../dat/Unionize_bias/SubSampleSib/"
topNs = list(range(200, 5, -5))

In [ ]:
ipsi_case_profile = SI_Rank_profile_Case(ASD_BiasDF, 
                    adj_mat_ipsi, ProbMat1_ipsi, ProbMat2_ipsi, topNs)
ipsi_ctrl_profile = SI_Rank_Profile_Ctrl(contDir, 
                    adj_mat_ipsi, ProbMat1_ipsi, ProbMat2_ipsi, topNs)

In [ ]:
plt.plot(topNs, ipsi_case_profile)
plt.plot(topNs, ipsi_ctrl_profile.mean(axis=0))

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(8,3))
cont_xx = ipsi_case_profile - ipsi_ctrl_profile
cont = np.nanmean(ipsi_ctrl_profile, axis=0)
mean = cont_xx.mean(axis=0)
lower = np.percentile(cont_xx, 50-34.1, axis=0)
upper = np.percentile(cont_xx, 50+34.1, axis=0)
ax.errorbar(topNs, mean, color="grey", marker="o", markersize=1.5, lw=0.5,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="Info-Diff")

ax.grid(True)
ax.set_xlabel("Structure Rank")
ax.set_ylabel("Circuit Score")
plt.legend()

In [ ]:
topN = 46

Ncont = 10000
contDir = "../dat/Unionize_bias/SubSampleSib/"

def TestPAtSize(ASD_BiasDF, contDir, topN, adj_mat, ProbMat1, ProbMat2):
    topN_bias = []
    cont_topN_Scores = []

    for i, file in enumerate(os.listdir(contDir)):
        if not file.startswith("cont.bias."):
            continue
        df = pd.read_csv(contDir + file, index_col="STR")
        top_strs = df.index.values[:topN]

        cont_score = ScoreCircuit_SI_ipsi_contra(top_strs, adj_mat, 
                                            ProbMat1, ProbMat2)
        cont_topN_Scores.append(cont_score)

        if i > Ncont:
            break
    asd_score = ScoreCircuit_SI_ipsi_contra(
        ASD_BiasDF.head(topN).index.values, adj_mat, ProbMat1, ProbMat2)
    return asd_score, cont_topN_Scores

In [ ]:
topN = 46
asd_score, cont_topN_Scores = TestPAtSize(
    ASD_BiasDF, contDir, topN, adj_mat_ipsi, ProbMat1_ipsi, ProbMat2_ipsi)

In [ ]:
fig, axs = plt.subplots(1,1, dpi=120, figsize=(3,3))
PlotPermutationP(cont_topN_Scores, asd_score, axs,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

### Contra

In [ ]:
adj_mat_contra = pd.read_csv(
    "../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat.Weight.contra.csv",
                          index_col=0)
InfoMat_contra = pd.read_csv(
    "../dat/allen-mouse-conn/ScoreingMat_jw_v2/ScoreMat_Distance.Entropy.contra.csv",
                     index_col=0)
ProbMat1_contra = np.exp2(-InfoMat_ipsi)
ProbMat1_contra[ProbMat1_contra==1] = 0
ProbMat2_contra = 1-ProbMat1_contra

In [ ]:
contra_case_profile = SI_Rank_profile_Case(ASD_BiasDF, 
                    adj_mat_contra, ProbMat1_contra, ProbMat2_contra, topNs)
contra_ctrl_profile = SI_Rank_Profile_Ctrl(contDir, 
                    adj_mat_contra, ProbMat1_contra, ProbMat2_contra, topNs)

In [ ]:
plt.plot(topNs, contra_case_profile)
plt.plot(topNs, contra_ctrl_profile.mean(axis=0))

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(8,3))
cont_xx = contra_case_profile - contra_ctrl_profile
cont = np.nanmean(contra_ctrl_profile, axis=0)
mean = cont_xx.mean(axis=0)
lower = np.percentile(cont_xx, 50-34.1, axis=0)
upper = np.percentile(cont_xx, 50+34.1, axis=0)
ax.errorbar(topNs, mean, color="grey", marker="o", markersize=1.5, lw=0.5,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="Info-Diff")

ax.grid(True)
ax.set_xlabel("Structure Rank")
ax.set_ylabel("Circuit Score")
plt.legend()

In [ ]:
topN = 46
asd_score, cont_topN_Scores = TestPAtSize(
    ASD_BiasDF, contDir, topN, adj_mat_contra, ProbMat1_contra, ProbMat2_contra)
fig, axs = plt.subplots(1,1, dpi=120, figsize=(3,3))
PlotPermutationP(cont_topN_Scores, asd_score, axs,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

### Combined

In [ ]:
adj_mat = adj_mat_ipsi.join(adj_mat_contra, how='left')
#InfoMat = InfoMat_ipsi.join(InfoMat_contra, how='left')
InfoMat = InfoMat_contra
ProbMat1 = np.exp2(-InfoMat)
ProbMat1[ProbMat1==1] = 0
ProbMat2 = 1-ProbMat1

In [ ]:
case_profile = SI_Rank_profile_Case(ASD_BiasDF, 
                    adj_mat, ProbMat1, ProbMat2, topNs)
ctrl_profile = SI_Rank_Profile_Ctrl(contDir, 
                    adj_mat, ProbMat1, ProbMat2, topNs)

In [ ]:
plt.plot(topNs, case_profile)
plt.plot(topNs, ctrl_profile.mean(axis=0))

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(8,3))
cont_xx = case_profile - ctrl_profile
cont = np.nanmean(contra_ctrl_profile, axis=0)
mean = cont_xx.mean(axis=0)
lower = np.percentile(cont_xx, 50-34.1, axis=0)
upper = np.percentile(cont_xx, 50+34.1, axis=0)
ax.errorbar(topNs, mean, color="grey", marker="o", markersize=1.5, lw=0.5,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="Info-Diff")

ax.grid(True)
ax.set_xlabel("Structure Rank")
ax.set_ylabel("Circuit Score")
plt.legend()

In [ ]:
topN = 46
asd_score, cont_topN_Scores = TestPAtSize(
    ASD_BiasDF, contDir, topN, adj_mat, ProbMat1, ProbMat2)
fig, axs = plt.subplots(1,1, dpi=120, figsize=(3,3))
PlotPermutationP(cont_topN_Scores, asd_score, axs,
                     title="Info Per Edge Inside Circuit".format(), xlabel="Normed Score", 
                     dist_label="Sibling", bar_label="ASD")

### Circuit Distance

In [ ]:
# Load Data and bin graphs
#ASD_STRs = pd.read_csv("dat/bias2/ASD.EW.pLI.z2.csv").head(50)["STR"].values
#ASD_STRs = pd.read_csv("dat/bias2/New.ASD.EW.pLI.expL.csv").head(50)["STR"].values
ASD_STRs = pd.read_csv("dat/bias2/New.ASD.EW.Z2.qn.nn.csv").head(30)["STR"].values
Sib_STRs = pd.read_csv("dat/bias2/ASD.Siblings.z2.csv").head(30)["STR"].values

adj_mat = pd.read_csv(ConnFil, index_col=0)
graph = LoadConnectome2()

Cartesian_distances_w_edge = MaskDistMat(adj_mat, Cartesian_distancesDF, cutoff=0)
Distance_Cuts = [0, 1000, 2000, 3000, 4000, 5000, 100000]
Dist_cut_graphs = []

N_Connections_total = []
for i, cut in enumerate(Distance_Cuts[:-1]):
    Conn_mat_new, distance_mat_new = MaskDistMat_xx(Cartesian_distances_w_edge, adj_mat, keep="bw",
                                                cutoff=Distance_Cuts[i], cutoff2=Distance_Cuts[i+1])
    N_Connections_total.append(np.count_nonzero(Conn_mat_new))
    g_ = LoadConnectome2(Conn_mat_new)
    Dist_cut_graphs.append(g_)


### XX

In [ ]:
def MaskDistMat(Mat1, Mat2, cutoff, m='lt'):
    New_Mat2 = Mat2.copy(deep=True)
    for STR_i in Mat1.index.values:
        for STR_j in Mat1.columns.values:
            if m == 'gt':
                if Mat1.loc[STR_i, STR_j] >= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
            elif m == "lt":
                if Mat1.loc[STR_i, STR_j] <= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
    return New_Mat2

def MaskDistMat_xx(distance_mat, Conn_mat, cutoff, cutoff2, keep='gt'):
    Conn_mat_new = Conn_mat.copy(deep=True)
    distance_mat_new = distance_mat.copy(deep=True)
    for STR_i in distance_mat.index.values:
        for STR_j in distance_mat.columns.values:
            if keep == 'gt':
                if distance_mat.loc[STR_i, STR_j] >= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0
            elif keep == "lt":
                if distance_mat.loc[STR_i, STR_j] <= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0   
            elif keep=="bw":
                if distance_mat.loc[STR_i, STR_j] >= cutoff and distance_mat.loc[STR_i, STR_j] <= cutoff2:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0   
    return Conn_mat_new, distance_mat_new

In [ ]:
# Load Data and bin graphs
#ASD_STRs = pd.read_csv("dat/bias2/ASD.EW.pLI.z2.csv").head(50)["STR"].values
#ASD_STRs = pd.read_csv("dat/bias2/New.ASD.EW.pLI.expL.csv").head(50)["STR"].values
ASD_STRs = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.v2.csv").head(50)["STR"].values
Sib_STRs = pd.read_csv("../dat/Unionize_bias/sib.61.Z2.bias.FDR.v2.csv").head(50)["STR"].values

adj_mat = adj_mat_ipsi
graph = LoadConnectome2(adj_mat)

Cartesian_distances_w_edge = MaskDistMat(adj_mat, DistanceMat_ipsi, cutoff=0)
Distance_Cuts = [0, 1000, 2000, 3000, 4000, 5000, 100000]
Dist_cut_graphs = []

N_Connections_total = []
for i, cut in enumerate(Distance_Cuts[:-1]):
    Conn_mat_new, distance_mat_new = MaskDistMat_xx(Cartesian_distances_w_edge, adj_mat, keep="bw",
                                                cutoff=Distance_Cuts[i], cutoff2=Distance_Cuts[i+1])
    N_Connections_total.append(np.count_nonzero(Conn_mat_new))
    g_ = LoadConnectome2(Conn_mat_new)
    Dist_cut_graphs.append(g_)

In [ ]:
ASD_top_conn, Sib_top_conn = [], []
N_Connections_total = []
for i,v in enumerate(Dist_cut_graphs):
    g_ = Dist_cut_graphs[i]
    N_Connections_total.append(len(g_.es))
    cohe, Nconn = ScoreSTRSet(g_, ASD_STRs)
    ASD_top_conn.append(Nconn)
    cohe, Nconn = ScoreSTRSet(g_, Sib_STRs)
    Sib_top_conn.append(Nconn)
ASD_top_conn = np.array(ASD_top_conn)
Sib_top_conn = np.array(Sib_top_conn)
N_Connections_total = np.array(N_Connections_total)

asd_conn_density = ASD_top_conn/N_Connections_total
sib_conn_density = Sib_top_conn/N_Connections_total

In [ ]:
def getratio_asd_sib(Dist_cut_graphs, ASD_STRs, Sib_STRs):
    ASD_top49_conn, Sib_top49_conn = [], []
    N_Connections_total = []
    for i,v in enumerate(Dist_cut_graphs):
        g_ = Dist_cut_graphs[i]
        N_Connections_total.append(len(g_.es))
        ASD_top49_conn.append(len(subgraph(g_, ASD_STRs).es))
        Sib_top49_conn.append(len(subgraph(g_, Sib_STRs).es))
    ASD_top49_conn = np.array(ASD_top49_conn)
    Sib_top49_conn = np.array(Sib_top49_conn)
    N_Connections_total = np.array(N_Connections_total)
    return ASD_top49_conn, Sib_top49_conn, N_Connections_total

Nboot = 1000
Dat_ASD_boots, Dat_Sib_boots = [], []
for i in range(Nboot):
    ASD_STR_boot = np.random.choice(ASD_STRs, replace=True, size=len(ASD_STRs))
    Sib_STR_boot = np.random.choice(Sib_STRs, replace=True, size=len(Sib_STRs))
    g2_asd = subgraph(graph, ASD_STR_boot)
    g2_sib = subgraph(graph, Sib_STR_boot)
    bl_asd = len(g2_asd.es)/np.count_nonzero(adj_mat)
    bl_sib = len(g2_sib.es)/np.count_nonzero(adj_mat)
    ASD_top_conn, Sib_top_conn, N_Connections_total = getratio_asd_sib(
        Dist_cut_graphs, ASD_STR_boot, Sib_STR_boot)
    dat_asd = ASD_top_conn/N_Connections_total
    dat_sib = Sib_top_conn/N_Connections_total
    Dat_ASD_boots.append(dat_asd)
    Dat_Sib_boots.append(dat_sib)

Dat_ASD_boots = np.array(Dat_ASD_boots)
Dat_Sib_boots = np.array(Dat_Sib_boots)

ASD_Dy, Sib_Dy = [], []
for i in range(6):
    boots = Dat_ASD_boots[:, i]
    ASD_Dy.append(np.std(boots))
    
    boots = Dat_Sib_boots[:, i]
    Sib_Dy.append(np.std(boots))

ASD_Dy = np.array(ASD_Dy)
ASD_Dy = ASD_Dy.transpose()

Sib_Dy = np.array(Sib_Dy)
Sib_Dy = Sib_Dy.transpose()

In [ ]:
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import matplotlib.ticker as mticker  
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 25})
fig, ax1 = plt.subplots(dpi=480, figsize=(12,6))

ax1.errorbar(np.arange(6), asd_conn_density, yerr=ASD_Dy, fmt=".", 
             label="Proband Connections", color="blue")
ax1.errorbar(np.arange(6) + 0.05, sib_conn_density, yerr=Sib_Dy, fmt=".k", 
             label="Sibings Connections", color="orange")
#ax1.errorbar(np.arange(6) , rand_mean, yerr=Rand_Dy, fmt=".k", 
#             label="Random Connections", color="gray")
ax1.plot(np.arange(6), asd_conn_density, marker="." , color="blue")
ax1.plot(np.arange(6) + 0.05, sib_conn_density, marker=".", color="orange")
#ax1.plot(np.arange(6)  , rand_mean, marker=".", color="gray", ls="dashed")


ax1.grid(True)
ax1.legend(loc="upper right", fontsize=20)
ax1.set_ylabel("Connections Density", fontsize=25)
ax1.set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax1.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax1.tick_params(axis='y', labelsize=20)
#plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%s x1000'))
plt.tight_layout()
#fig.savefig("figs/n_Fig_4.A.pdf")

In [ ]:
adj_mat = adj_mat_contra
graph = LoadConnectome2(adj_mat)

Cartesian_distances_w_edge = MaskDistMat(adj_mat, DistanceMat_contra, cutoff=0)
Distance_Cuts = [0, 1000, 2000, 3000, 4000, 5000, 100000]
Dist_cut_graphs = []

N_Connections_total = []
for i, cut in enumerate(Distance_Cuts[:-1]):
    Conn_mat_new, distance_mat_new = MaskDistMat_xx(Cartesian_distances_w_edge, adj_mat, keep="bw",
                                                cutoff=Distance_Cuts[i], cutoff2=Distance_Cuts[i+1])
    N_Connections_total.append(np.count_nonzero(Conn_mat_new))
    g_ = LoadConnectome2(Conn_mat_new)
    Dist_cut_graphs.append(g_)

In [ ]:
ASD_top_conn, Sib_top_conn = [], []
N_Connections_total = []
for i,v in enumerate(Dist_cut_graphs):
    g_ = Dist_cut_graphs[i]
    N_Connections_total.append(len(g_.es))
    cohe, Nconn = ScoreSTRSet(g_, ASD_STRs)
    ASD_top_conn.append(Nconn)
    cohe, Nconn = ScoreSTRSet(g_, Sib_STRs)
    Sib_top_conn.append(Nconn)
ASD_top_conn = np.array(ASD_top_conn)
Sib_top_conn = np.array(Sib_top_conn)
N_Connections_total = np.array(N_Connections_total)

asd_conn_density = ASD_top_conn/N_Connections_total
sib_conn_density = Sib_top_conn/N_Connections_total

In [ ]:
def getratio_asd_sib(Dist_cut_graphs, ASD_STRs, Sib_STRs):
    ASD_top49_conn, Sib_top49_conn = [], []
    N_Connections_total = []
    for i,v in enumerate(Dist_cut_graphs):
        g_ = Dist_cut_graphs[i]
        N_Connections_total.append(len(g_.es))
        ASD_top49_conn.append(len(subgraph(g_, ASD_STRs).es))
        Sib_top49_conn.append(len(subgraph(g_, Sib_STRs).es))
    ASD_top49_conn = np.array(ASD_top49_conn)
    Sib_top49_conn = np.array(Sib_top49_conn)
    N_Connections_total = np.array(N_Connections_total)
    return ASD_top49_conn, Sib_top49_conn, N_Connections_total

Nboot = 1000
Dat_ASD_boots, Dat_Sib_boots = [], []
for i in range(Nboot):
    ASD_STR_boot = np.random.choice(ASD_STRs, replace=True, size=len(ASD_STRs))
    Sib_STR_boot = np.random.choice(Sib_STRs, replace=True, size=len(Sib_STRs))
    g2_asd = subgraph(graph, ASD_STR_boot)
    g2_sib = subgraph(graph, Sib_STR_boot)
    bl_asd = len(g2_asd.es)/np.count_nonzero(adj_mat)
    bl_sib = len(g2_sib.es)/np.count_nonzero(adj_mat)
    ASD_top_conn, Sib_top_conn, N_Connections_total = getratio_asd_sib(
        Dist_cut_graphs, ASD_STR_boot, Sib_STR_boot)
    dat_asd = ASD_top_conn/N_Connections_total
    dat_sib = Sib_top_conn/N_Connections_total
    Dat_ASD_boots.append(dat_asd)
    Dat_Sib_boots.append(dat_sib)

Dat_ASD_boots = np.array(Dat_ASD_boots)
Dat_Sib_boots = np.array(Dat_Sib_boots)

ASD_Dy, Sib_Dy = [], []
for i in range(6):
    boots = Dat_ASD_boots[:, i]
    ASD_Dy.append(np.std(boots))
    
    boots = Dat_Sib_boots[:, i]
    Sib_Dy.append(np.std(boots))

ASD_Dy = np.array(ASD_Dy)
ASD_Dy = ASD_Dy.transpose()

Sib_Dy = np.array(Sib_Dy)
Sib_Dy = Sib_Dy.transpose()

In [ ]:
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import matplotlib.ticker as mticker  
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 25})
fig, ax1 = plt.subplots(dpi=480, figsize=(12,6))

ax1.errorbar(np.arange(6), asd_conn_density, yerr=ASD_Dy, fmt=".", 
             label="Proband Connections", color="blue")
ax1.errorbar(np.arange(6) + 0.05, sib_conn_density, yerr=Sib_Dy, fmt=".k", 
             label="Sibings Connections", color="orange")
#ax1.errorbar(np.arange(6) , rand_mean, yerr=Rand_Dy, fmt=".k", 
#             label="Random Connections", color="gray")
ax1.plot(np.arange(6), asd_conn_density, marker="." , color="blue")
ax1.plot(np.arange(6) + 0.05, sib_conn_density, marker=".", color="orange")
#ax1.plot(np.arange(6)  , rand_mean, marker=".", color="gray", ls="dashed")


ax1.grid(True)
ax1.legend(loc="upper right", fontsize=20)
ax1.set_ylabel("Connections Density", fontsize=25)
ax1.set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax1.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax1.tick_params(axis='y', labelsize=20)
#plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%s x1000'))
plt.tight_layout()
#fig.savefig("figs/n_Fig_4.A.pdf")